In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms.v2
from torchvision.io import read_image
from torch.utils.data.dataloader import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import tv_tensors

import math
import glob
import os
import random
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
import xml.etree.ElementTree as ET

torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
dataset_params = {
  "train_im_sets": ["data/VOC2007"],
  "test_im_sets": ["data/VOC2007-test"],
  "num_classes": 21,
  "im_size": 300
}

model_params = {
  "im_channels": 3,
  "aspect_ratios": [
    [ 1., 2., 0.5 ],
    [ 1., 2., 3., 0.5, .333 ],
    [ 1., 2., 3., 0.5, .333 ],
    [ 1., 2., 3., 0.5, .333 ],
    [ 1., 2., 0.5 ],
    [ 1., 2., 0.5 ]
  ],
  "scales": [0.1, 0.2, 0.375, 0.55, 0.725, 0.9],
  "iou_threshold": 0.5,
  "low_score_threshold": 0.01,
  "neg_pos_ratio": 3,
  "pre_nms_topK": 400,
  "detections_per_img": 200,
  "nms_threshold": 0.45
}

train_params = {
  "task_name": "voc",
  "seed": 1111,
  "acc_steps": 1, # Increase if you want to get gradients from >1 steps (kind of mimicking >1 batch size)
  "num_epochs": 100,
  "batch_size": 8,
  "lr_steps": [40, 50, 60, 70, 80, 90],
  "lr": 0.001,
  "log_steps": 100,
  "infer_conf_threshold": 0.5,
  "ckpt_name": "ssd_voc2007.pth"
}

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
!unzip -q "/content/drive/MyDrive/Machine Learning/advanced-deep-learning/object-detection/YOLO/data.zip" -d "/content"

In [5]:
def load_images_and_anns(im_sets, label2idx, ann_fname, split):
  """
  Method to get the xml files and for each file get all the objects and their ground truth detection information for the dataset

  im_sets: Sets of images to consider
  label2idx: Class Name to index mapping for dataset
  ann_fname: txt file containing image names{trainval.txt/test.txt}
  split: train/test
  """
  im_infos = []
  ims = []

  for im_set in im_sets:
    im_names = []
    # Fetch all image names in txt file for this imageset
    for line in open(os.path.join(im_set, "ImageSets", "Main", f"{ann_fname}.txt")):
      im_names.append(line.strip())

    # Set annotation and image path
    ann_dir = os.path.join(im_set, "Annotations")
    im_dir = os.path.join(im_set, "JPEGImages")

    for im_name in im_names:
      ann_file = os.path.join(ann_dir, f"{im_name}.xml")
      im_info = {}
      ann_info = ET.parse(ann_file)
      root = ann_info.getroot()
      size = root.find("size")

      width = int(size.find("width").text)
      height = int(size.find("height").text)

      im_info["img_id"] = os.path.basename(ann_file).split(".xml")[0]
      im_info["filename"] = os.path.join(im_dir, f"{im_info['img_id']}.jpg")
      im_info["width"] = width
      im_info["height"] = height
      detections = []

      for obj in ann_info.findall("object"):
        det = {}
        label = label2idx[obj.find("name").text]
        difficult = int(obj.find("difficult").text)
        bbox_info = obj.find("bndbox")

        bbox = [
            int(bbox_info.find("xmin").text) - 1,
            int(bbox_info.find("ymin").text) - 1,
            int(bbox_info.find("xmax").text) - 1,
            int(bbox_info.find("ymax").text) - 1
        ]

        det["label"] = label
        det["bbox"] = bbox
        det["difficult"] = difficult

        # At test time eval does the job of ignoring difficult
        detections.append(det)

      im_info["detections"] = detections
      im_infos.append(im_info)

  print(f"Total {len(im_infos)} images found")
  return im_infos

In [6]:
class VOCDataset(Dataset):
  def __init__(self, split, im_sets, im_size=300):
    self.split = split

    # Imagesets for this dataset instance (VOC2007/VOC2007+VOC2012/VOC2007-test)
    self.im_sets = im_sets
    self.fname = "trainval" if self.split == "train" else "test"
    self.im_size = im_size
    self.im_mean = [123.0, 117.0, 104.0]
    self.imagenet_mean = [0.485, 0.456, 0.406]
    self.imagenet_std = [0.229, 0.224, 0.225]

    # Train and test transformations
    self.transforms = {
        "train": torchvision.transforms.v2.Compose([
            torchvision.transforms.v2.RandomPhotometricDistort(),
            torchvision.transforms.v2.RandomZoomOut(fill=self.im_mean),
            torchvision.transforms.v2.RandomIoUCrop(),
            torchvision.transforms.v2.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.v2.Resize(size=(self.im_size, self.im_size)),
            torchvision.transforms.v2.SanitizeBoundingBoxes(
                labels_getter=lambda transform_input:
                (transform_input[1]["labels"], transform_input[1]["difficult"])),
            torchvision.transforms.v2.ToPureTensor(),
            torchvision.transforms.v2.ToDtype(torch.float32, scale=True),
            torchvision.transforms.v2.Normalize(mean=self.imagenet_mean,
                                                std=self.imagenet_std)

        ]),
        "test": torchvision.transforms.v2.Compose([
            torchvision.transforms.v2.Resize(size=(self.im_size, self.im_size)),
            torchvision.transforms.v2.ToPureTensor(),
            torchvision.transforms.v2.ToDtype(torch.float32, scale=True),
            torchvision.transforms.v2.Normalize(mean=self.imagenet_mean,
                                                std=self.imagenet_std)
        ]),
    }

    classes = [
        "person", "bird", "cat", "cow", "dog", "horse", "sheep",
        "aeroplane", "bicycle", "boat", "bus", "car", "motorbike", "train",
        "bottle", "chair", "diningtable", "pottedplant", "sofa", "tvmonitor"
    ]
    classes = sorted(classes)
    # We need to add background class as well with 0 index
    classes = ["background"] + classes

    self.label2idx = {classes[idx]: idx for idx in range(len(classes))}
    self.idx2label = {idx: classes[idx] for idx in range(len(classes))}
    print(self.idx2label)
    self.images_info = load_images_and_anns(self.im_sets,
                                            self.label2idx,
                                            self.fname,
                                            self.split)

  def __len__(self):
    return len(self.images_info)

  def __getitem__(self, index):
    im_info = self.images_info[index]
    im = read_image(im_info["filename"])

    # Get annotations for this image
    targets = {}
    targets["bboxes"] = tv_tensors.BoundingBoxes(
        [detection["bbox"] for detection in im_info["detections"]],
        format="XYXY", canvas_size=im.shape[-2:])
    targets["labels"] = torch.as_tensor(
        [detection["label"] for detection in im_info["detections"]])
    targets["difficult"] = torch.as_tensor(
        [detection["difficult"]for detection in im_info["detections"]])

    # Transform the image and targets
    transformed_info = self.transforms[self.split](im, targets)
    im_tensor, targets = transformed_info

    h, w = im_tensor.shape[-2:]

    wh_tensor = torch.as_tensor([[w, h, w, h]]).expand_as(targets["bboxes"])
    targets["bboxes"] = targets["bboxes"] / wh_tensor
    return im_tensor, targets, im_info["filename"]

In [7]:
def get_iou(boxes1, boxes2):
  """
  IOU between two sets of boxes
  boxes1: (Tensor of shape N x 4)
  boxes2: (Tensor of shape M x 4)
  """

  # Area of boxes (x_2 - x_1) * (y_2 - y_1)
  area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])  # (N,)
  area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])  # (M,)

  # Get top left x_1, y_1 coordinate
  x_left = torch.max(boxes1[:, None, 0], boxes2[:, 0])  # (N, M)
  y_top = torch.max(boxes1[:, None, 1], boxes2[:, 1])  # (N, M)

  # Get bottom right x_2, y_2 coordinate
  x_right = torch.min(boxes1[:, None, 2], boxes2[:, 2])  # (N, M)
  y_bottom = torch.min(boxes1[:, None, 3], boxes2[:, 3])  # (N, M)

  intersection_area = ((x_right - x_left).clamp(min=0) *
                        (y_bottom - y_top).clamp(min=0))  # (N, M)
  union = area1[:, None] + area2 - intersection_area  # (N, M)
  iou = intersection_area / union  # (N, M)

  return iou

In [8]:
def boxes_to_transformation_targets(ground_truth_boxes,
                                    default_boxes,
                                    weights=(10., 10., 5., 5.)):
  """
  Method to compute targets for each default_boxes.
  Assumes boxes are in x1y1x2y2 format.
  We first convert boxes to cx,cy,w,h format and then
  compute targets based on following formulation

  target_dx = (gt_cx - default_boxes_cx) / default_boxes_w
  target_dy = (gt_cy - default_boxes_cy) / default_boxes_h
  target_dw = log(gt_w / default_boxes_w)
  target_dh = log(gt_h / default_boxes_h)

  ground_truth_boxes: (Tensor of shape N x 4)
  default_boxes: (Tensor of shape N x 4)
  weights: Tuple[float] -> (wx, wy, ww, wh)
  """

  # Get center_x, center_y, width, height from x_1, y_1, x_2, y_2 for default_boxes
  widths = default_boxes[:, 2] - default_boxes[:, 0]
  heights = default_boxes[:, 3] - default_boxes[:, 1]
  center_x = default_boxes[:, 0] + 0.5 * widths
  center_y = default_boxes[:, 1] + 0.5 * heights

  # # Get center_x, center_y, width, height from x_1, y_1, x_2, y_2 for ground-truth boxes
  gt_widths = (ground_truth_boxes[:, 2] - ground_truth_boxes[:, 0])
  gt_heights = ground_truth_boxes[:, 3] - ground_truth_boxes[:, 1]
  gt_center_x = ground_truth_boxes[:, 0] + 0.5 * gt_widths
  gt_center_y = ground_truth_boxes[:, 1] + 0.5 * gt_heights

  # Use formulation to compute all targets
  targets_dx = weights[0] * (gt_center_x - center_x) / widths
  targets_dy = weights[1] * (gt_center_y - center_y) / heights
  targets_dw = weights[2] * torch.log(gt_widths / widths)
  targets_dh = weights[3] * torch.log(gt_heights / heights)
  regression_targets = torch.stack((targets_dx,
                                    targets_dy,
                                    targets_dw,
                                    targets_dh), dim=1)
  return regression_targets

In [9]:
def apply_regression_pred_to_default_boxes(box_transform_pred,
                                           default_boxes,
                                           weights=(10., 10., 5., 5.)):
  """
  Method to transform default_boxes based on transformation parameter
  prediction.
  Assumes boxes are in x_1, y_1, x_2, y_2 format

  box_transform_pred: (Tensor of shape N x 4)
  default_boxes: (Tensor of shape N x 4)
  weights: Tuple[float] -> (wx, wy, ww, wh)
  """

  # Get cx, cy, w, h from x1,y1,x2,y2
  w = default_boxes[:, 2] - default_boxes[:, 0]
  h = default_boxes[:, 3] - default_boxes[:, 1]
  center_x = default_boxes[:, 0] + 0.5 * w
  center_y = default_boxes[:, 1] + 0.5 * h

  dx = box_transform_pred[..., 0] / weights[0]
  dy = box_transform_pred[..., 1] / weights[1]
  dw = box_transform_pred[..., 2] / weights[2]
  dh = box_transform_pred[..., 3] / weights[3]
  # dh -> (num_default_boxes)

  pred_center_x = dx * w + center_x
  pred_center_y = dy * h + center_y
  pred_w = torch.exp(dw) * w
  pred_h = torch.exp(dh) * h
  # pred_center_x -> (num_default_boxes, 4)

  pred_box_x1 = pred_center_x - 0.5 * pred_w
  pred_box_y1 = pred_center_y - 0.5 * pred_h
  pred_box_x2 = pred_center_x + 0.5 * pred_w
  pred_box_y2 = pred_center_y + 0.5 * pred_h

  pred_boxes = torch.stack((
      pred_box_x1,
      pred_box_y1,
      pred_box_x2,
      pred_box_y2),
      dim=-1)
  return pred_boxes

In [10]:
def generate_default_boxes(feat, aspect_ratios, scales):
  """
  Method to generate default_boxes for all feature maps the image

  feat: List[(Tensor of shape B x C x Feat_H x Feat x W)]
  aspect_ratios: List[List[float]] aspect ratios for each feature map
  scales: List[float] scales for each feature map
  """

  # List to store default boxes for all feature maps
  default_boxes = []
  for k in range(len(feat)):
    # We first add the aspect ratio 1 and scale (sqrt(scale[k])*sqrt(scale[k+1])
    s_prime_k = math.sqrt(scales[k] * scales[k + 1])
    wh_pairs = [[s_prime_k, s_prime_k]]

    # Adding all possible w,h pairs according to
    # aspect ratio of the feature map k
    for ar in aspect_ratios[k]:
      sq_ar = math.sqrt(ar)
      w = scales[k] * sq_ar
      h = scales[k] / sq_ar

      wh_pairs.extend([[w, h]])

    feat_h, feat_w = feat[k].shape[-2:]

    # These shifts will be the centre of each of the default boxes
    shifts_x = ((torch.arange(0, feat_w) + 0.5) / feat_w).to(torch.float32)
    shifts_y = ((torch.arange(0, feat_h) + 0.5) / feat_h).to(torch.float32)
    shift_y, shift_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")
    shift_x = shift_x.reshape(-1)
    shift_y = shift_y.reshape(-1)

    # Duplicate these shifts for as many boxes (aspect ratios) per position we have
    shifts = torch.stack((shift_x, shift_y) * len(wh_pairs), dim=-1).reshape(-1, 2)
    # shifts for first feature map will be (5776 x 2)

    wh_pairs = torch.as_tensor(wh_pairs)

    # Repeat the wh pairs for all positions in feature map
    wh_pairs = wh_pairs.repeat((feat_h * feat_w), 1)
    # wh_pairs for first feature map will be (5776 x 2)

    # Concat the shifts(cx cy) and wh values for all positions
    default_box = torch.cat((shifts, wh_pairs), dim=1)
    # default box for feat_1 -> (5776, 4)
    # default box for feat_2 -> (2166, 4)
    # default box for feat_3 -> (600, 4)
    # default box for feat_4 -> (150, 4)
    # default box for feat_5 -> (36, 4)
    # default box for feat_6 -> (4, 4)

    default_boxes.append(default_box)

  default_boxes = torch.cat(default_boxes, dim=0)
  # default_boxes -> (8732, 4)

  # We now duplicate these default boxes
  # for all images in the batch
  # and also convert cx,cy,w,h format of
  # default boxes to x1,y1,x2,y2
  dboxes = []
  for _ in range(feat[0].size(0)):
    dboxes_in_image = default_boxes
    # x1 = cx - 0.5 * width
    # y1 = cy - 0.5 * height
    # x2 = cx + 0.5 * width
    # y2 = cy + 0.5 * height
    dboxes_in_image = torch.cat(
        [
            (dboxes_in_image[:, :2] - 0.5 * dboxes_in_image[:, 2:]),
            (dboxes_in_image[:, :2] + 0.5 * dboxes_in_image[:, 2:]),
        ],
        -1,
    )
    dboxes.append(dboxes_in_image.to(feat[0].device))

  return dboxes

In [15]:
class SSD(nn.Module):
  """
  Main Class for SSD. Does the following steps to generate detections/losses.
  During initialization
  1. Load VGG Imagenet pretrained model
  2. Extract Backbone from VGG and add extra conv layers
  3. Add class prediction and bbox transformation prediction layers
  4. Initialize all conv2d layers

  During Forward Pass
  1. Get conv4_3 output
  2. Normalize and scale conv4_3 output (feat_output_1)
  3. Pass the unscaled conv4_3 to conv5_3 layers and conv layers
      replacing fc6 and fc7 of vgg (feat_output_2)
  4. Pass the conv_fc7 output to extra conv layers (feat_output_3-6)
  5. Get the classification and regression predictions for all 6 feature maps
  6. Generate default_boxes for all these feature maps(8732 x 4)
  7a. If in training assign targets for these default_boxes and
      compute localization and classification losses
  7b. If in inference mode, then do all pre-nms filtering, nms
      and then post nms filtering and return the detected boxes,
      their labels and their scores
  """
  def __init__(self, config, num_classes=21):
    super().__init__()

    self.aspect_ratios = config["aspect_ratios"]

    self.scales = config["scales"]
    self.scales.append(1.0)

    self.num_classes = num_classes
    self.iou_threshold = config["iou_threshold"]
    self.low_score_threshold = config["low_score_threshold"]
    self.neg_pos_ratio = config["neg_pos_ratio"]
    self.pre_nms_topK = config["pre_nms_topK"]
    self.nms_threshold = config["nms_threshold"]
    self.detections_per_img = config["detections_per_img"]

    # Load imagenet pretrained vgg network
    backbone = torchvision.models.vgg16(
        weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1
    )

    # Get all max pool indexes to determine different stages
    max_pool_pos = [idx for idx, layer in enumerate(list(backbone.features))
                    if isinstance(layer, nn.MaxPool2d)]
    max_pool_stage_3_pos = max_pool_pos[-3]  # for vgg16 this would be 16
    max_pool_stage_4_pos = max_pool_pos[-2]  # for vgg16 this would be 23

    backbone.features[max_pool_stage_3_pos].ceil_mode = True

    # otherwise vgg conv4_3 output will be 37x37
    self.features = nn.Sequential(*backbone.features[:max_pool_stage_4_pos])
    self.scale_weight = nn.Parameter(torch.ones(512) * 20)

    # Conv5_3 + Conv for fc6 and fc 7
    # Conv modules replacing fc6 and fc7
    # Ideally we would copy the weights, but here we are just adding new layers and not copying fc6 and fc7 weights by subsampling
    fcs = nn.Sequential(
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        nn.Conv2d(in_channels=512,
                  out_channels=1024,
                  kernel_size=3,
                  stride=1,
                  padding=6,
                  dilation=6),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=1024,
                  out_channels=1024,
                  kernel_size=1,
                  stride=1,
                  padding=0),
        nn.ReLU(inplace=True),
    )
    self.conv5_3_fc = nn.Sequential(
        *backbone.features[max_pool_stage_4_pos:-1],
        fcs,
    )

    # Additional Conv Layers

    # Modules to take from 19x19 to 10x10
    self.conv8_2 = nn.Sequential(
        nn.Conv2d(in_channels=1024,
                  out_channels=256,
                  kernel_size=1,
                  stride=1,
                  padding=0),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=256,
                  out_channels=512,
                  kernel_size=3,
                  stride=2,
                  padding=1),
        nn.ReLU(inplace=True)
    )

    # Modules to take from 10x10 to 5x5
    self.conv9_2 = nn.Sequential(
        nn.Conv2d(in_channels=512,
                  out_channels=128,
                  kernel_size=1,
                  stride=1,
                  padding=0),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=128,
                  out_channels=256,
                  kernel_size=3,
                  stride=2,
                  padding=1),
        nn.ReLU(inplace=True)
    )

    # Modules to take from 5x5 to 3x3
    self.conv10_2 = nn.Sequential(
        nn.Conv2d(256, 128, kernel_size=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(128, 256, kernel_size=3),
        nn.ReLU(inplace=True)
    )

    # Modules to take from 3x3 to 1x1
    self.conv11_2 = nn.Sequential(
        nn.Conv2d(in_channels=256,
                  out_channels=128,
                  kernel_size=1,
                  stride=1,
                  padding=0),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=128,
                  out_channels=256,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(inplace=True)
    )

    # Must match conv4_3, fcs, conv8_2, conv9_2, conv10_2, conv11_2
    out_channels = [512, 1024, 512, 256, 256, 256]

    # Prediction Layers
    self.cls_heads = nn.ModuleList()
    for channels, aspect_ratio in zip(out_channels, self.aspect_ratios):
      # extra 1 is added for scale of sqrt(sk * sk + 1)
      self.cls_heads.append(nn.Conv2d(channels,
                                      self.num_classes * (len(aspect_ratio) + 1),
                                      kernel_size=3,
                                      padding=1))

    self.bbox_reg_heads = nn.ModuleList()
    for channels, aspect_ratio in zip(out_channels, self.aspect_ratios):
      # extra 1 is added for scale of sqrt(sk * sk + 1)
      self.bbox_reg_heads.append(nn.Conv2d(channels, 4 * (len(aspect_ratio)+1),
                                            kernel_size=3,
                                            padding=1))

    # Conv Layer Initialization
    for layer in fcs.modules():
        if isinstance(layer, nn.Conv2d):
            torch.nn.init.xavier_uniform_(layer.weight)
            if layer.bias is not None:
                torch.nn.init.constant_(layer.bias, 0.0)

    for conv_module in [self.conv8_2, self.conv9_2, self.conv10_2, self.conv11_2]:
        for layer in conv_module.modules():
            if isinstance(layer, nn.Conv2d):
                torch.nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    torch.nn.init.constant_(layer.bias, 0.0)

    for module in self.cls_heads:
        torch.nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            torch.nn.init.constant_(module.bias, 0.0)
    for module in self.bbox_reg_heads:
        torch.nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            torch.nn.init.constant_(module.bias, 0.0)

  def compute_loss(self,
                   targets,
                   cls_logits,
                   bbox_regression,
                   default_boxes,
                   matched_idxs):
    # Counting all the foreground default_boxes for computing N in loss equation
    num_foreground = 0
    # BBox losses for all batch images(for foreground default_boxes)
    bbox_loss = []
    # classification targets for all batch images(for ALL default_boxes)
    cls_targets = []
    for (
        targets_per_image,
        bbox_regression_per_image,
        cls_logits_per_image,
        default_boxes_per_image,
        matched_idxs_per_image,
    ) in zip(targets, bbox_regression, cls_logits, default_boxes, matched_idxs):
      # Foreground default_boxes -> matched_idx >=0
      # Background default_boxes -> matched_idx = -1
      fg_idxs_per_image = torch.where(matched_idxs_per_image >= 0)[0]
      foreground_matched_idxs_per_image = matched_idxs_per_image[
          fg_idxs_per_image
      ]
      num_foreground += foreground_matched_idxs_per_image.numel()

      # Get foreground default_boxes and their transformation predictions
      matched_gt_boxes_per_image = targets_per_image["boxes"][
          foreground_matched_idxs_per_image
      ]
      bbox_regression_per_image = bbox_regression_per_image[fg_idxs_per_image, :]
      default_boxes_per_image = default_boxes_per_image[fg_idxs_per_image, :]
      target_regression = boxes_to_transformation_targets(
          matched_gt_boxes_per_image,
          default_boxes_per_image)

      bbox_loss.append(
          torch.nn.functional.smooth_l1_loss(bbox_regression_per_image,
                                              target_regression,
                                              reduction='sum')
      )

      # Get classification target for ALL default_boxes
      # For all default_boxes set it as 0 first
      # Then set foreground default_boxes target as label
      # of assigned gt box
      gt_classes_target = torch.zeros(
          (cls_logits_per_image.size(0),),
          dtype=targets_per_image["labels"].dtype,
          device=targets_per_image["labels"].device,
      )
      gt_classes_target[fg_idxs_per_image] = targets_per_image["labels"][
          foreground_matched_idxs_per_image
      ]
      cls_targets.append(gt_classes_target)

    # Aggregated bbox loss and classification targets
    # for all batch images
    bbox_loss = torch.stack(bbox_loss)
    cls_targets = torch.stack(cls_targets)  # (B, 8732)

    # Calculate classification loss for ALL default_boxes
    num_classes = cls_logits.size(-1)
    cls_loss = torch.nn.functional.cross_entropy(cls_logits.view(-1, num_classes),
                                                  cls_targets.view(-1),
                                                  reduction="none").view(
        cls_targets.size()
    )

    # Hard Negative Mining
    foreground_idxs = cls_targets > 0
    # We will sample total of 3 x (number of fg default_boxes)
    # background default_boxes
    num_negative = self.neg_pos_ratio * foreground_idxs.sum(1, keepdim=True)

    # As of now cls_loss is for ALL default_boxes
    negative_loss = cls_loss.clone()
    # We want to ensure that after sorting based on loss value,
    # foreground default_boxes are never picked when choosing topK
    # highest loss indexes
    negative_loss[foreground_idxs] = -float("inf")
    values, idx = negative_loss.sort(1, descending=True)
    # Fetch those indexes which have in topK(K=num_negative) losses
    background_idxs = idx.sort(1)[1] < num_negative
    N = max(1, num_foreground)
    return {
        "bbox_regression": bbox_loss.sum() / N,
        "classification": (cls_loss[foreground_idxs].sum() +
                            cls_loss[background_idxs].sum()) / N,
    }

  def forward(self, x, targets=None):
    # Call everything till conv4_3 layers first
    conv_4_3_out = self.features(x)

    # Scale conv4_3 output using learnt norm scale
    conv_4_3_out_scaled = (self.scale_weight.view(1, -1, 1, 1) *
                            torch.nn.functional.normalize(conv_4_3_out))

    # Call conv5_3 with non_scaled conv_3 and also
    # Call additional conv layers
    conv_5_3_fc_out = self.conv5_3_fc(conv_4_3_out)
    conv8_2_out = self.conv8_2(conv_5_3_fc_out)
    conv9_2_out = self.conv9_2(conv8_2_out)
    conv10_2_out = self.conv10_2(conv9_2_out)
    conv11_2_out = self.conv11_2(conv10_2_out)

    # Feature maps for predictions
    outputs = [
        conv_4_3_out_scaled,  # 38 x 38
        conv_5_3_fc_out,  # 19 x 19
        conv8_2_out,  # 10 x 10
        conv9_2_out,  # 5 x 5
        conv10_2_out,  # 3 x 3
        conv11_2_out,   # 1 x 1
    ]

    # Classification and bbox regression for all feature maps
    cls_logits = []
    bbox_reg_deltas = []
    for i, features in enumerate(outputs):
      cls_feat_i = self.cls_heads[i](features)
      bbox_reg_feat_i = self.bbox_reg_heads[i](features)

      # Cls output from (B, A * num_classes, H, W) to (B, HWA, num_classes).
      N, _, H, W = cls_feat_i.shape
      cls_feat_i = cls_feat_i.view(N, -1, self.num_classes, H, W)
      # (B, A, num_classes, H, W)
      cls_feat_i = cls_feat_i.permute(0, 3, 4, 1, 2)  # (B, H, W, A, num_classes)
      cls_feat_i = cls_feat_i.reshape(N, -1, self.num_classes)
      # (B, HWA, num_classes)
      cls_logits.append(cls_feat_i)

      # Permute bbox reg output from (B, A * 4, H, W) to (B, HWA, 4).
      N, _, H, W = bbox_reg_feat_i.shape
      bbox_reg_feat_i = bbox_reg_feat_i.view(N, -1, 4, H, W)  # (B, A, 4, H, W)
      bbox_reg_feat_i = bbox_reg_feat_i.permute(0, 3, 4, 1, 2)  # (B, H, W, A, 4)
      bbox_reg_feat_i = bbox_reg_feat_i.reshape(N, -1, 4)  # Size=(B, HWA, 4)
      bbox_reg_deltas.append(bbox_reg_feat_i)

    # Concat cls logits and bbox regression predictions for all feature maps
    cls_logits = torch.cat(cls_logits, dim=1)  # (B, 8732, num_classes)
    bbox_reg_deltas = torch.cat(bbox_reg_deltas, dim=1)  # (B, 8732, 4)

    # Generate default_boxes for all feature maps
    default_boxes = generate_default_boxes(outputs, self.aspect_ratios, self.scales)
    # default_boxes -> List[Tensor of shape 8732 x 4]
    # len(default_boxes) = Batch size

    losses = {}
    detections = []
    if self.training:
      # List to hold for each image, which default box
      # is assigned to with gt box if any
      # or unassigned(background)
      matched_idxs = []
      for default_boxes_per_image, targets_per_image in zip(default_boxes, targets):
        if targets_per_image["boxes"].numel() == 0:
            matched_idxs.append(
                torch.full(
                    (default_boxes_per_image.size(0),), -1,
                    dtype=torch.int64,
                    device=default_boxes_per_image.device
                )
            )
            continue

        iou_matrix = get_iou(targets_per_image["boxes"],
                              default_boxes_per_image)
        # For each default box find best ground truth box
        matched_vals, matches = iou_matrix.max(dim=0)
        # matches -> [8732]

        # Update index of match for all default_boxes which
        # have maximum iou with a gt box < low threshold
        # as -1
        # This allows selecting foreground boxes as match index >= 0
        below_low_threshold = matched_vals < self.iou_threshold
        matches[below_low_threshold] = -1

        # We want to also assign the best default box for every gt
        # as foreground
        # So first find the best default box for every gt
        _, highest_quality_pred_foreach_gt = iou_matrix.max(dim=1)
        # Update the best matching gt index for these best default_boxes
        # as 0, 1, 2, ...., len(gt)-1
        matches[highest_quality_pred_foreach_gt] = torch.arange(
            highest_quality_pred_foreach_gt.size(0), dtype=torch.int64,
            device=highest_quality_pred_foreach_gt.device
        )
        matched_idxs.append(matches)

      losses = self.compute_loss(targets, cls_logits, bbox_reg_deltas,
                                  default_boxes, matched_idxs)
    else:
      # For test time we do the following:
      # 1. Convert default_boxes to boxes using predicted bbox regression deltas
      # 2. Low score filtering
      # 3. Pre-NMS TopK filtering
      # 4. NMS
      # 5. Post NMS TopK Filtering
      cls_scores = torch.nn.functional.softmax(cls_logits, dim=-1)
      num_classes = cls_scores.size(-1)

      for bbox_deltas_i, cls_scores_i, default_boxes_i in zip(bbox_reg_deltas,
                                                            cls_scores,
                                                            default_boxes):
        boxes = apply_regression_pred_to_default_boxes(bbox_deltas_i,
                                                        default_boxes_i)
        # Ensure all values are between 0-1
        boxes.clamp_(min=0., max=1.)

        pred_boxes = []
        pred_scores = []
        pred_labels = []
        # Class wise filtering
        for label in range(1, num_classes):
          score = cls_scores_i[:, label]

          # Remove low scoring boxes of this class
          keep_idxs = score > self.low_score_threshold
          score = score[keep_idxs]
          box = boxes[keep_idxs]

          # keep only topk scoring predictions of this class
          score, top_k_idxs = score.topk(min(self.pre_nms_topK, len(score)))
          box = box[top_k_idxs]

          pred_boxes.append(box)
          pred_scores.append(score)
          pred_labels.append(torch.full_like(score, fill_value=label,
                                              dtype=torch.int64,
                                              device=cls_scores.device))

        pred_boxes = torch.cat(pred_boxes, dim=0)
        pred_scores = torch.cat(pred_scores, dim=0)
        pred_labels = torch.cat(pred_labels, dim=0)

        # Class wise NMS
        keep_mask = torch.zeros_like(pred_scores, dtype=torch.bool)
        for class_id in torch.unique(pred_labels):
          curr_indices = torch.where(pred_labels == class_id)[0]
          curr_keep_idxs = torch.ops.torchvision.nms(pred_boxes[curr_indices],
                                                      pred_scores[curr_indices],
                                                      self.nms_threshold)
          keep_mask[curr_indices[curr_keep_idxs]] = True

        keep_indices = torch.where(keep_mask)[0]
        post_nms_keep_indices = keep_indices[pred_scores[keep_indices].sort(
            descending=True)[1]]
        keep = post_nms_keep_indices[:self.detections_per_img]
        pred_boxes, pred_scores, pred_labels = (pred_boxes[keep],
                                                pred_scores[keep],
                                                pred_labels[keep])

        detections.append(
            {
                "boxes": pred_boxes,
                "scores": pred_scores,
                "labels": pred_labels,
            }
        )

    return losses, detections

In [16]:
def collate_function(data):
  return tuple(zip(*data))

In [21]:
def train():
  dataset_config = dataset_params
  train_config = train_params

  # Seet the random seed
  seed = train_config["seed"]
  torch.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
  if device == "cuda":
    torch.cuda.manual_seed_all(seed)

  voc = VOCDataset("train",
                    im_sets=dataset_config["train_im_sets"],
                    im_size=dataset_config["im_size"])

  train_dataset = DataLoader(voc,
                              batch_size=train_config["batch_size"],
                              shuffle=True,
                              collate_fn=collate_function)

  # Instantiate model and load checkpoint if present
  model = SSD(config=model_params, num_classes=dataset_config["num_classes"])
  model.to(device)
  model.train()

  if not os.path.exists(train_config["task_name"]):
    os.mkdir(train_config["task_name"])

  optimizer = torch.optim.SGD(lr=train_config["lr"],
                              params=model.parameters(),
                              weight_decay=5E-4, momentum=0.9)
  lr_scheduler = MultiStepLR(optimizer, milestones=train_config["lr_steps"], gamma=0.5)

  acc_steps = train_config["acc_steps"]
  num_epochs = train_config["num_epochs"]
  steps = 0

  # Resume training from a saved checkpoint if it exists
  checkpoint_path = os.path.join(train_config["task_name"], train_config["ckpt_name"])
  start_epoch = 0

  if os.path.exists(checkpoint_path):
    print("Resuming training from checkpoint...")
    checkpoint = torch.load(checkpoint_path, map_location=device)

    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    lr_scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    start_epoch = checkpoint["epoch"]
    steps = checkpoint["steps"]

  optimizer.zero_grad()

  for i in range(start_epoch, num_epochs):
    ssd_classification_losses = []
    ssd_localization_losses = []
    for idx, (ims, targets, _) in enumerate(tqdm(train_dataset)):
      for target in targets:
        target["boxes"] = target["bboxes"].float().to(device)
        del target["bboxes"]
        target["labels"] = target["labels"].long().to(device)

      images = torch.stack([im.float().to(device) for im in ims], dim=0)
      batch_losses, _ = model(images, targets)
      loss = batch_losses["classification"]
      loss += batch_losses["bbox_regression"]

      ssd_classification_losses.append(batch_losses["classification"].item())
      ssd_localization_losses.append(batch_losses["bbox_regression"].item())
      loss = loss / acc_steps
      loss.backward()

      if (idx + 1) % acc_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
      if steps % train_config["log_steps"] == 0:
        loss_output = ""
        loss_output += f"SSD Classification Loss: {np.mean(ssd_classification_losses):.4f}"
        loss_output += f" | SSD Localization Loss: {np.mean(ssd_localization_losses):.4f}"
        print(loss_output)
      if torch.isnan(loss):
        print("Loss is becoming nan. Exiting")
        exit(0)

      steps += 1

    optimizer.step()
    optimizer.zero_grad()
    lr_scheduler.step()
    print("Finished epoch {}".format(i+1))
    loss_output = ""
    loss_output += f"SSD Classification Loss: {np.mean(ssd_classification_losses):.4f}"
    loss_output += f" | SSD Localization Loss: {np.mean(ssd_localization_losses):.4f}"
    print(loss_output)

    checkpoint = {
        "epoch": i,
        "steps": steps,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": lr_scheduler.state_dict(),
    }
    torch.save(checkpoint, os.path.join(train_config["task_name"], train_config["ckpt_name"]))

  print("Done Training...")

train()

{0: 'background', 1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat', 5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair', 10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse', 14: 'motorbike', 15: 'person', 16: 'pottedplant', 17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor'}
Total 5011 images found


  0%|          | 1/627 [00:00<01:33,  6.73it/s]

SSD Classification Loss: 26.4580 | SSD Localization Loss: 3.1259


 16%|█▋        | 102/627 [00:22<02:10,  4.01it/s]

SSD Classification Loss: 14.9343 | SSD Localization Loss: 2.9918


...

SSD Classification Loss: 2.8304 | SSD Localization Loss: 0.8849


 84%|████████▍ | 528/627 [01:59<00:20,  4.85it/s]

SSD Classification Loss: 2.8231 | SSD Localization Loss: 0.8833


100%|██████████| 627/627 [02:22<00:00,  4.41it/s]


Finished epoch 100
SSD Classification Loss: 2.8299 | SSD Localization Loss: 0.8888
Done Training...


In [22]:
def get_iou_map(det, gt):
  det_x1, det_y1, det_x2, det_y2 = det
  gt_x1, gt_y1, gt_x2, gt_y2 = gt

  x_left = max(det_x1, gt_x1)
  y_top = max(det_y1, gt_y1)
  x_right = min(det_x2, gt_x2)
  y_bottom = min(det_y2, gt_y2)

  if x_right < x_left or y_bottom < y_top:
    return 0.0

  area_intersection = (x_right - x_left) * (y_bottom - y_top)
  det_area = (det_x2 - det_x1) * (det_y2 - det_y1)
  gt_area = (gt_x2 - gt_x1) * (gt_y2 - gt_y1)

  area_union = float(det_area + gt_area - area_intersection + 1E-6)
  iou = area_intersection / area_union

  return iou

In [23]:
def compute_map(det_boxes, gt_boxes, iou_threshold=0.5, method="area", difficult=None):
  gt_labels = {cls_key for im_gt in gt_boxes for cls_key in im_gt.keys()}
  gt_labels = sorted(gt_labels)

  all_aps = {}

  # Average precisions for ALL classes
  aps = []
  for idx, label in enumerate(gt_labels):
    # Get detection predictions of this class
    cls_dets = [
        [im_idx, im_dets_label] for im_idx, im_dets in enumerate(det_boxes)
        if label in im_dets for im_dets_label in im_dets[label]
    ]

    # Sort them by confidence score
    cls_dets = sorted(cls_dets, key=lambda k: -k[1][-1])

    # For tracking which gt boxes of this class have already been matched
    gt_matched = [[False for _ in im_gts[label]] for im_gts in gt_boxes]
    # Number of gt boxes for this class for recall calculation
    num_gts = sum([len(im_gts[label]) for im_gts in gt_boxes])
    num_difficults = sum([sum(difficults_label[label]) for difficults_label in difficult])

    tp = [0] * len(cls_dets)
    fp = [0] * len(cls_dets)

    # For each prediction
    for det_idx, (im_idx, det_pred) in enumerate(cls_dets):
      # Get gt boxes for this image and this label
      im_gts = gt_boxes[im_idx][label]
      im_gt_difficults = difficult[im_idx][label]

      max_iou_found = -1
      max_iou_gt_idx = -1

      # Get best matching gt box
      for gt_box_idx, gt_box in enumerate(im_gts):
        gt_box_iou = get_iou_map(det_pred[:-1], gt_box)
        if gt_box_iou > max_iou_found:
          max_iou_found = gt_box_iou
          max_iou_gt_idx = gt_box_idx
      # TP only if iou >= threshold and this gt has not yet been matched
      if max_iou_found >= iou_threshold:
        if not im_gt_difficults[max_iou_gt_idx]:
          if not gt_matched[im_idx][max_iou_gt_idx]:
            # If tp then we set this gt box as matched
            gt_matched[im_idx][max_iou_gt_idx] = True
            tp[det_idx] = 1
          else:
            fp[det_idx] = 1
      else:
        fp[det_idx] = 1

    # Cumulative tp and fp
    tp = np.cumsum(tp)
    fp = np.cumsum(fp)

    eps = np.finfo(np.float32).eps
    # recalls = tp / np.maximum(num_gts, eps)
    recalls = tp / np.maximum(num_gts - num_difficults, eps)
    precisions = tp / np.maximum((tp + fp), eps)

    if method == "area":
      recalls = np.concatenate(([0.0], recalls, [1.0]))
      precisions = np.concatenate(([0.0], precisions, [0.0]))

      # Replace precision values with recall r with maximum precision value
      # of any recall value >= r
      # This computes the precision envelope
      for i in range(precisions.size - 1, 0, -1):
          precisions[i - 1] = np.maximum(precisions[i - 1], precisions[i])
      # For computing area, get points where recall changes value
      i = np.where(recalls[1:] != recalls[:-1])[0]
      # Add the rectangular areas to get ap
      ap = np.sum((recalls[i + 1] - recalls[i]) * precisions[i + 1])
    elif method == "interp":
      ap = 0.0
      for interp_pt in np.arange(0, 1 + 1E-3, 0.1):
        # Get precision values for recall values >= interp_pt
        prec_interp_pt = precisions[recalls >= interp_pt]

        # Get max of those precision values
        prec_interp_pt= prec_interp_pt.max() if prec_interp_pt.size>0.0 else 0.0
        ap += prec_interp_pt
      ap = ap / 11.0
    else:
      raise ValueError("Method can only be area or interp")
    if num_gts > 0:
      aps.append(ap)
      all_aps[label] = ap
    else:
      all_aps[label] = np.nan

  # compute mAP at provided iou threshold
  mean_ap = sum(aps) / len(aps)
  return mean_ap, all_aps

In [24]:
def load_model_and_dataset():
  dataset_config = dataset_params
  model_config = model_params
  train_config = train_params

  voc = VOCDataset("test", im_sets=dataset_config["test_im_sets"])

  test_dataset = DataLoader(voc,
                            batch_size=1,
                            shuffle=False,
                            num_workers=os.cpu_count(),
                            pin_memory=True)

  model = SSD(config=model_config, num_classes=dataset_config["num_classes"])

  model.to(device)
  model.eval()

  checkpoint = torch.load(os.path.join(train_config["task_name"], train_config["ckpt_name"]),
                          map_location=device)

  model.load_state_dict(checkpoint["model_state_dict"])

  return model, voc, test_dataset

In [25]:
def infer():
  if not os.path.exists("samples"):
    os.mkdir("samples")

  model, voc, test_dataset = load_model_and_dataset()
  conf_threshold = train_params["infer_conf_threshold"]
  model.low_score_threshold = conf_threshold

  num_samples = 5

  for i in tqdm(range(num_samples)):
    dataset_idx = random.randint(0, len(voc) - 1)
    im_tensor, target, fname = voc[dataset_idx]
    _, ssd_detections = model(im_tensor.unsqueeze(0).to(device), [target])

    gt_im = cv2.imread(fname)
    h, w = gt_im.shape[:2]
    gt_im_copy = gt_im.copy()

    # Saving images with ground truth boxes
    for idx, box in enumerate(target["bboxes"]):
      x1, y1, x2, y2 = box.detach().cpu().numpy()
      x1, y1, x2, y2 = int(w*x1), int(h*y1), int(w*x2), int(h*y2)
      cv2.rectangle(gt_im, (x1, y1), (x2, y2), thickness=2, color=[0, 255, 0])
      cv2.rectangle(gt_im_copy, (x1, y1), (x2, y2), thickness=2, color=[0, 255, 0])
      text = voc.idx2label[target["labels"][idx].detach().cpu().item()]
      text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
      text_w, text_h = text_size
      cv2.rectangle(gt_im_copy, (x1, y1), (x1 + 10 + text_w, y1 + 10 + text_h), [255, 255, 255], -1)
      cv2.putText(gt_im, text=voc.idx2label[target["labels"][idx].detach().cpu().item()],
                  org=(x1 + 5, y1 + 15),
                  thickness=1,
                  fontScale=1,
                  color=[0, 0, 0],
                  fontFace=cv2.FONT_HERSHEY_PLAIN)
      cv2.putText(gt_im_copy, text=text,
                  org=(x1 + 5, y1 + 15),
                  thickness=1,
                  fontScale=1,
                  color=[0, 0, 0],
                  fontFace=cv2.FONT_HERSHEY_PLAIN)

    cv2.addWeighted(gt_im_copy, 0.7, gt_im, 0.3, 0, gt_im)
    cv2.imwrite(f"samples/output_ssd_gt_{i}.png", gt_im)

    # Getting predictions from trained model
    boxes = ssd_detections[0]["boxes"]
    labels = ssd_detections[0]["labels"]
    scores = ssd_detections[0]["scores"]
    im = cv2.imread(fname)
    im_copy = im.copy()

    # Saving images with predicted boxes
    for idx, box in enumerate(boxes):
      x1, y1, x2, y2 = box.detach().cpu().numpy()
      x1, y1, x2, y2 = int(w * x1), int(h * y1), int(w * x2), int(h * y2)
      cv2.rectangle(im, (x1, y1), (x2, y2), thickness=2, color=[0, 0, 255])
      cv2.rectangle(im_copy, (x1, y1), (x2, y2), thickness=2, color=[0, 0, 255])
      text = f"{voc.idx2label[labels[idx].detach().cpu().item()]}: {scores[idx].detach().cpu().item():.2f}"

      text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
      text_w, text_h = text_size
      cv2.rectangle(im_copy, (x1, y1), (x1 + 10 + text_w, y1 + 10 + text_h), [255, 255, 255], -1)
      cv2.putText(im, text=text,
                  org=(x1 + 5, y1 + 15),
                  thickness=1,
                  fontScale=1,
                  color=[0, 0, 0],
                  fontFace=cv2.FONT_HERSHEY_PLAIN)
      cv2.putText(im_copy, text=text,
                  org=(x1 + 5, y1 + 15),
                  thickness=1,
                  fontScale=1,
                  color=[0, 0, 0],
                  fontFace=cv2.FONT_HERSHEY_PLAIN)

    cv2.addWeighted(im_copy, 0.7, im, 0.3, 0, im)

    cv2.imwrite(f"samples/output_ssd_{i}.jpg", im)

  print("Done Detecting...")

infer()

{0: 'background', 1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat', 5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair', 10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse', 14: 'motorbike', 15: 'person', 16: 'pottedplant', 17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor'}
Total 4952 images found


100%|██████████| 5/5 [00:00<00:00,  7.96it/s]

Done Detecting...


In [26]:
def evaluate_map():
  model, voc, test_dataset = load_model_and_dataset()

  gts = []
  preds = []
  difficults = []
  for im_tensor, target, fname in tqdm(test_dataset):
    im_tensor = im_tensor.float().to(device)
    target_bboxes = target["bboxes"].float()[0].to(device)
    target_labels = target["labels"].long()[0].to(device)
    difficult = target["difficult"].long()[0].to(device)
    _, ssd_detections = model(im_tensor)

    boxes = ssd_detections[0]["boxes"]
    labels = ssd_detections[0]["labels"]
    scores = ssd_detections[0]["scores"]

    pred_boxes = {}
    gt_boxes = {}
    difficult_boxes = {}

    for label_name in voc.label2idx:
        pred_boxes[label_name] = []
        gt_boxes[label_name] = []
        difficult_boxes[label_name] = []

    for idx, box in enumerate(boxes):
      x1, y1, x2, y2 = box.detach().cpu().numpy()
      label = labels[idx].detach().cpu().item()
      score = scores[idx].detach().cpu().item()
      label_name = voc.idx2label[label]
      pred_boxes[label_name].append([x1, y1, x2, y2, score])

    for idx, box in enumerate(target_bboxes):
      x1, y1, x2, y2 = box.detach().cpu().numpy()
      label = target_labels[idx].detach().cpu().item()
      label_name = voc.idx2label[label]
      gt_boxes[label_name].append([x1, y1, x2, y2])
      difficult_boxes[label_name].append(difficult[idx].detach().cpu().item())

    gts.append(gt_boxes)
    preds.append(pred_boxes)
    difficults.append(difficult_boxes)

  mean_ap, all_aps = compute_map(preds, gts, method="area", difficult=difficults)
  print("Class Wise Average Precisions")
  for idx in range(len(voc.idx2label)):
    print(f"AP for class {voc.idx2label[idx]} = {all_aps[voc.idx2label[idx]]:.4f}")
  print(f"Mean Average Precision: {mean_ap:.4f}")

evaluate_map()

{0: 'background', 1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat', 5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair', 10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse', 14: 'motorbike', 15: 'person', 16: 'pottedplant', 17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor'}
Total 4952 images found


100%|██████████| 4952/4952 [02:47<00:00, 29.64it/s]


Class Wise Average Precisions
AP for class background = nan
AP for class aeroplane = 0.7674
AP for class bicycle = 0.8246
AP for class bird = 0.7120
AP for class boat = 0.6578
AP for class bottle = 0.3309
AP for class bus = 0.8131
AP for class car = 0.8568
AP for class cat = 0.8576
AP for class chair = 0.4494
AP for class cow = 0.7227
AP for class diningtable = 0.6930
AP for class dog = 0.8373
AP for class horse = 0.8482
AP for class motorbike = 0.8297
AP for class person = 0.7537
AP for class pottedplant = 0.3838
AP for class sheep = 0.6980
AP for class sofa = 0.7492
AP for class train = 0.8526
AP for class tvmonitor = 0.7187
Mean Average Precision: 0.7178


In [27]:
!zip -r samples.zip samples

  adding: samples/ (stored 0%)
  adding: samples/output_ssd_2.jpg (deflated 0%)
  adding: samples/output_ssd_gt_2.png (deflated 1%)
  adding: samples/output_ssd_gt_0.png (deflated 1%)
  adding: samples/output_ssd_0.jpg (deflated 0%)
  adding: samples/output_ssd_gt_1.png (deflated 3%)
  adding: samples/output_ssd_3.jpg (deflated 0%)
  adding: samples/output_ssd_4.jpg (deflated 1%)
  adding: samples/output_ssd_gt_3.png (deflated 2%)
  adding: samples/output_ssd_gt_4.png (deflated 2%)
  adding: samples/output_ssd_1.jpg (deflated 0%)


In [28]:
from google.colab import files

files.download("samples.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>